# Project: Modeling gravitational lensing by a black hole

In **gravitational lensing**, light from a distant source (like a galaxy or a quasar) is deflected by the gravitational field of a massive intervening object (like a black hole) lying along our line of sight. This deflection can produce multiple images, magnifications, and distortions of the background source. Originally predicted by Einstein’s theory of general relativity, this phenomenon has become a powerful tool in astronomy, informing studies of dark matter distribution, galaxy cluster mass profiles, and the large-scale structure of the universe.

In this project, you'll investigate a simple case of gravitational lensing by modeling a point-mass lens, analogous to a single black hole. Through theoretical derivations and Python-based analysis, you will explore how to calculate image positions, magnifications, and the lensing of an extended source. An optional extension allows you to incorporate effects from a spinning (Kerr) black hole to see how image positions shift under different spin parameters.

---

## Model

Throughout this project, we'll adopt two approximations. First, we'll assume that the lens mass is concentrated in a point-like object (like an idealized black hole). Second, we'll assume that the bending of light occurs in a thin plane at the lens location (the so-called thin-lens approximation). Together, these assumptions allow us to dedine the simplified 1D **lens equation**:

$\beta = \theta - \frac{\theta_E^2}{\theta}$

where:
- $\beta$ is the angular position of the source (relative to the optical axis).
- $\theta$ is the angular position of the lensed image.
- $\theta_E$ is the Einstein radius, a characteristic angular scale in gravitational lensing that depends on the lens mass and the distances involved.

You can think of $\theta$ and $\beta$ as angles on the sky, measured in radians. Both are defined relative to the optical axis, which is the imaginary line connecting the observer to the center of the lens mass. For a more detailed derivation of all of these quantities, see [here](https://www.haus-der-astronomie.de/3440781/gravitational_lensing_brems.pdf).

The Einstein radius $\theta_E$ is given by:

$\theta_E = \sqrt{\frac{4GM}{c^2} \frac{D_{LS}}{D_L D_S}}$

where:
- $G$ is the gravitational constant.
- $c$ is the speed of light.
- $M$ is the lens mass.
- $D_S$ is the distance between the source and the observer.
- $D_L$ is the distance between the lens and the observer.
- $D_{LS}$ is the distance between the lens and the source.

This project is primarily theoretical, but you might find it helpful to look up typical black hole masses or distances to known lens systems for reference. Example masses could be:

- $M_{\rm BH} \sim 10^6 - 10^9 M_{\odot}$ (for supermassive black holes)
- $M_{\rm BH} \sim 3 - 10 M_{\odot}$ (for stellar-mass black holes)

You can substitute realistic values for $D_L$ and $D_S$ by referencing typical distances (e.g. parsecs to kiloparsecs). You can also just choose convenient numbers if you like.

---

## Analysis tasks 

### 1. Compute the Einstein radius for a generic system
Write a function `einstein_radius(mass, D_LS, D_L, D_S)` that computes the Einstein radius $\theta_E$ for any set of input parameters.

### 2. Solve the lens equation for a generic system

#### 2a. Derive analytic solutions
The point-mass lens equation is $\beta = \theta - \frac{\theta_E^2}{\theta}$. Rearrange this to get a quadratic equation in $\theta$, and solve for $\theta$. Write your derivation in a markdown cell.

#### 2b. Implement Python function
Once you have the analytic solutions, write a function `solve_lens_equation(beta, theta_E)` that implements these solutions to calculate the image positions ($\theta_+$ and $\theta_-$) given a source position $\beta$ (assume $\beta$ is small, in radians) and the Einstein radius $\theta_E$.

#### 2c. Verify alignment scenario
Show that for exact alignment of the lens and source ($\beta = 0$), both image positions converge to $\pm\theta_E$ (or one of them converges to $\theta_E$ and the other to $-\theta_E$, depending on sign convention). Use your `solve_lens_equation` function to demonstrate that this is true.

### 3. Investigate the effect of lens mass on image separation
For a range of lens masses, calculate the separation between the images $|\theta_+ - \theta_-|$. Create a scatterplot showing how this separation changes as a function of lens mass. Note that the other parameters should be fixed to reasonable values, though you can repeat this investigation to determine their effects as well if you're curious!

### 4. Investigate magnification 
Write a function `magnification(theta, theta_E)` that computes the magnification of an image located at $\theta$, when the Einstein radius is $\theta_E$. For a point-mass lens,

$\mu_\pm(\theta) = \left\lvert \frac{1}{1 - \frac{\theta_E^4}{\theta_\pm^4}} \right\rvert$

Fix a lens mass and distances (hence fixing $\theta_E$). Vary $\beta$ over a range (e.g., from 0 to a few $\theta_E$) and plot how the magnification of each image changes.

### 5. Model lensing in 2D

In 2D, the point-mass lens equation becomes $\boldsymbol{\beta} = \boldsymbol{\theta} - \frac{\theta_E^2}{\lvert \boldsymbol{\theta} \rvert^2}$, where $\lvert \boldsymbol{\theta} \rvert = \sqrt{\theta_x^2 + \theta_y^2}$ (the magnitude of the image-plane projection vector). Here, bold symbols represent arrays of values. This means that you can apply the function to each dimension ($x$ and $y$) separately to get a transformed pair of coordinates.

Use the provided `make_fake_galaxy` function to generate a model "galaxy" source, with a brightness distribution represented by a 2D Gaussian. Using this model as the source plane, choose parameters for the lens mass and distances and map the "galaxy" to the image plane. 

A simple way to implement this is by using **backwards ray-tracing**. In this method, each point (or pixel) in the *image plane* is treated as the endpoint of a ray that originated in the *source plane*. Instead of starting from the source and projecting forward (which can be complicated due to multiple image locations), each light ray is traced backwards from our detector (the image plane) to find where it would have come from in the source plane. Once the source-plane location is known, the original source brightness at that location can be assigned to the image-plane pixel. By doing this for every pixel in the image plane, the “lensed” image can be constructed. 

<br>
<details>
    <summary>(Click <font color="red"><b>here</b></font> for a more in-depth hint/explanation of how to implement backwards ray-tracing in Python.)</summary>
  
1. **Set up the image-plane grid:** Define a 2D grid of $(\theta_x, \theta_y)$ values spanning the region of interest, where each grid cell represents a pixel in the lensed image. You will likely want to make this grid larger than the grid representing your source plane.

2. **Compute source-plane coordinates:** At each pixel $(\theta_x, \theta_y)$, calculate $\beta_x$ and $\beta_y$ using the lens equation $\boldsymbol{\beta}(\boldsymbol{\theta})$ . This “traces back” the light ray to its origin in the source plane. 

3. **Sample the source brightness:** Use the array $S(\beta_x, \beta_y)$ generated by the `make_fake_galaxy` function for the source’s intrinsic brightness distribution. For each pixel $(\theta_x, \theta_y)$, look up the brightness at the corresponding $(\beta_x, \beta_y)$. This will require figuring out what indices in the array $(\beta_x, \beta_y)$ correspond to!

4. **Construct the lensed image:** Assign the brightness value $S(\beta_x, \beta_y)$ to the image-plane pixel $(\theta_x, \theta_y)$. The final 2D array you build is the “lensed” view of your source.

</details>

Display both the source and image planes side-by-side with [`matplotlib.pyplot.imshow`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html). Explore how changing $M$, $\beta$, or $\theta_E$ affects your resulting image, and summarize the relationships you observe.

### 6. Incorporate black hole spin (optional)

When a black hole spins, its rotating spacetime geometry alters the paths of nearby light rays, causing an additional asymmetry in the lensing beyond the standard Schwarzschild case. The strength of this effect depends on the dimensionless spin parameter (ranging from 0 to 1) and the angle between the black hole’s spin axis and the photon’s incoming trajectory.

Accurate lensing calculations around a **spinning (Kerr) black hole** involve solving complicated geodesic equations in a rotating spacetime, which quickly becomes computationally and analytically intractable. By using a simpler toy model, we can capture the essential effects of spin without delving into the full complexity of exact general-relativistic ray-tracing.

For example, the 2D lens equation can be modified to something like:

$\boldsymbol{\beta} = \boldsymbol{\theta} - \left[(1 + \kappa\cos\psi) \frac{\theta_E^2}{\lvert \boldsymbol{\theta} \rvert^2}\right]$

where
- $\kappa$ is the dimensionless spin parameter (0 = not spinning, 1 = maximum spin), and
- $\psi$ is the angle of an incoming photon relative to the spin axis of the black hole

Adjust your 2D model from step 5 to incorporate this toy model, and demonstrate how a non-zero spin affects the resulting images. Also explore how the images shift for different $\psi$, including $\psi = 0$ (co-rotating), $\psi = \pi/2$ (orthogonal) and $\psi = \pi$ (counter-rotating).

---

## Reflection

Write a brief (1-2 paragraphs) interpretation of the results you found above. Link it back to your original research question and key concepts from your literature review. (For this project in particular, you might consider thinking about how what you've done would change if you can't assume that your lens is a point mass.)

Then, write a brief (1-2 paragraphs) reflection on the limitations of your analysis. Are there any caveats or assumptions in your analysis? Could more data or a different method provide more robust results?

---

## Extending your analysis (optional)

Are there additional aspects of the dataset that you’d like to explore? Do you have ideas for refining the methods used in this notebook? Or maybe you’ve noticed an interesting pattern in your results that raises new questions? If you answered yes to any of these questions, I encourage you to extend your analysis! Feel free to reach out to me via email or visit office hours to discuss your ideas. If you're interested in diving deeper but aren’t sure where to start, I’m also happy to brainstorm with you. This is a great opportunity to practice developing your own research questions and exploring a dataset in a way that interests you.

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def make_fake_galaxy(x0, y0, sigma=0.2, bound=2, n_points=200):
    '''
    Generates a fake galaxy source profile (round 2D Gaussian) on a square grid,
    covering a coordinate range from -bound to +bound in both x and y.
    
    x0, y0: Center of the Gaussian (source-plane coordinates)
    sigma: Width of the Gaussian
    bound: The upper and lower bounds of the grid
    n_points: The number of points to divide the grid into
    '''
    x = np.linspace(-bound, bound, n_points)
    y = np.linspace(-bound, bound, n_points)
    xx, yy = np.meshgrid(x, y)
    rr2 = (xx - x0)**2 + (yy - y0)**2
    source = np.exp(-0.5 * rr2 / sigma**2)
    return source

In [ ]:
#Demonstrating use of fake galaxy function
gal = make_fake_galaxy(0, 0)

#Displaying the simulated galaxy
#origin='lower' will tell imshow that the bottom left corner should correspond to [0,0] in the 2D array
#extent can be used to tell imshow what bounds to put on the image (otherwise it will display in pixel #s)
plt.imshow(gal, origin='lower', extent=[-2, 2, -2, 2])
plt.show()